In [1]:
import os
os.environ["OMP_NUM_THREADS"] = '12'

import torch
import os.path as osp
import GCL.losses as L
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import GCL.augmentors as A
import torch
import torch.nn.functional as F

from torch import nn, tensor
from tqdm import tqdm
from torch.optim import Adam
from GCL.eval import get_split, LREvaluator, SVMEvaluator
from GCL.models import SingleBranchContrast
from torch_geometric.nn import GATConv, GCNConv, GATv2Conv
from torch_geometric.nn.inits import uniform
from torch_geometric.datasets import Planetoid
from scipy.io import loadmat
from torch_geometric.data import Data
from GCL.models.contrast_model import WithinEmbedContrast

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_mutual_info_score


In [2]:
class GConv(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=2):
        super(GConv, self).__init__()
        self.act = nn.SELU()
        self.num_layers = num_layers
        self.norm = nn.BatchNorm1d(hidden_dim, momentum=0.01)
        self.layers = nn.ModuleList()
        self.layers.append(
            GATConv(input_dim, hidden_dim)
        )
        for _ in range(1, num_layers):
            self.layers.append(
                GATConv(hidden_dim, hidden_dim)
            )

    def forward(self, x, edge_index, edge_weight=None):
        z = x
        for i in range(self.num_layers - 1):
            z = self.layers[i](z, edge_index, edge_weight)
            z = self.act(z)
        z = self.layers[-1](z, edge_index, edge_weight)
        return z


class Encoder(nn.Module):
    def __init__(self, encoder, augmentor, hidden_dim=256, n_clusters=3, v=1):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.augmentor = augmentor
        self.register_buffer("epsilon", torch.FloatTensor([1e-12]))

        self.cluster_layer = nn.Parameter(torch.Tensor(n_clusters, hidden_dim))
        self.v = v
        torch.nn.init.xavier_normal_(self.cluster_layer.data)

    def forward(self, x, edge_index_dict, edge_weight=None):
        aug1, aug2 = self.augmentor
        zs = []
        z1s = []
        z2s = []
        for edge_index in edge_index_dict.values():
            x1, edge_index1, edge_weight1 = aug1(x, edge_index, edge_weight)
            # x2, edge_index2, edge_weight2 = aug2(x, edge_index, edge_weight)

            z = self.encoder(x, edge_index, edge_weight)

            z1 = self.encoder(x1, edge_index1, edge_weight1)
            # z2 = self.encoder(x2, edge_index2, edge_weight2)

            zs.append(z)
            z1s.append(z1)
            z2s.append(z)

        # z = zs[0] + zs[1]
        q = 1.0 / (1.0 + torch.sum(torch.pow(z.unsqueeze(1) - self.cluster_layer, 2), 2) / self.v)
        q = q.pow((self.v + 1.0) / 2.0)
        q = (q.t() / torch.sum(q, 1)).t()
        return z, z1s, z2s, q


In [3]:

def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.t() / weight.sum(1)).t()


def train(encoder_model, contrast_model, data, optimizer, p):
    encoder_model.train()
    optimizer.zero_grad()
    _, z1s, z2s, q = encoder_model(data.x, data.edge_index, data.edge_attr)
    loss = None
    for i in range(len(z1s)):
        if loss is None:
            loss = contrast_model(z1s[i], z2s[i])
        else:
            loss += contrast_model(z1s[i], z2s[i])
    kl_loss = F.kl_div(q.log(), p, reduction='batchmean')
    loss = 0.01 * loss + 100 * kl_loss

    loss.backward()
    optimizer.step()
    return loss.item(), kl_loss.item()


def test(encoder_model, data):
    encoder_model.eval()
    z, _, _, _ = encoder_model(data.x, data.edge_index, data.edge_attr)
    split = get_split(num_samples=z.size()[0], train_ratio=0.1, test_ratio=0.8)
    result = SVMEvaluator()(z, data.y, split)
    return result


device = torch.device('cuda')
path = osp.join(osp.pardir, 'datasets', 'ACM')

mat = loadmat(osp.join(path, 'ACM3025.mat'))
print(mat.keys())


dict_keys(['__header__', '__version__', '__globals__', 'PAP', 'PLP', 'PMP', 'PTP', 'feature', 'label', 'test_idx', 'train_idx', 'val_idx'])


In [4]:
print(len(mat['train_idx'][0]))

600


In [5]:
# edge_index = []
# for i in tqdm(range(len(mat['PLP']))):
#     for j in range(len(mat['PLP'])):
#         if mat['PAP'][i][j] == 1:
#             edge_index.append([i, j])

edge_index_dict = {}
for etype in ['PLP']:
    edge_index = []
    for i in tqdm(range(len(mat[etype]))):
        for j in range(len(mat[etype])):
            if mat[etype][i][j] == 1:
                edge_index.append([i, j])
    print(len(edge_index))
    edge_index = tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_index_dict[etype] = edge_index

100%|██████████| 3025/3025 [00:10<00:00, 290.55it/s]


2210761


In [6]:
x = tensor(mat['feature'], dtype=torch.float)
y = torch.argmax(tensor(mat['label']), -1)
data = Data(x=x, y=y, edge_index=edge_index_dict).to(device)


In [7]:
del mat


In [8]:

aug1 = A.Compose([A.EdgeRemoving(pe=0.5), A.FeatureMasking(pf=0.1)])
aug2 = A.Compose([A.EdgeRemoving(pe=0.5), A.FeatureMasking(pf=0.1)])

gconv = GConv(input_dim=data.num_features, hidden_dim=256, num_layers=2).to(device)
encoder_model = Encoder(encoder=gconv, augmentor=(aug1, aug2)).to(device)
print(encoder_model.parameters())
contrast_model = WithinEmbedContrast(loss=L.BarlowTwins()).to(device)

optimizer = Adam(encoder_model.parameters(), lr=5e-4)

losss = []
kl_losss = []
epoch = 1000

kmeans = KMeans(n_clusters=3, n_init=20)

with torch.no_grad():
    z, _, _, q = encoder_model(data.x, data.edge_index, data.edge_attr)
_ = kmeans.fit_predict(z.data.cpu().numpy())
encoder_model.cluster_layer.data = torch.tensor(kmeans.cluster_centers_).to(device)

with tqdm(total=epoch, desc='(T)') as pbar:
    for epoch in range(1, epoch + 1):
        if epoch % 1 == 0:
            # update_interval
            _, _, _, tmp_q = encoder_model(data.x, data.edge_index, data.edge_attr)
            tmp_q = tmp_q.data
            p = target_distribution(tmp_q)

        loss, kl_loss = train(encoder_model, contrast_model, data, optimizer, p)
        pbar.set_postfix({'loss': loss, 'kl_loss': kl_loss})
        pbar.update()
        losss.append(loss)
        kl_losss.append(kl_loss)

plt.plot(range(epoch), losss)
plt.plot(range(epoch), kl_losss)
plt.show()

# test_result = test(encoder_model, data)
# print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')


<generator object Module.parameters at 0x000001B328B56E40>


(T):   0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 2.11 GiB (GPU 0; 8.00 GiB total capacity; 4.46 GiB already allocated; 0 bytes free; 6.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

with torch.no_grad():
    encoder_model.eval()
    z, _, _, _ = encoder_model(data.x, data.edge_index, data.edge_attr)

pred = kmeans.fit_predict(z.cpu())

nmi = normalized_mutual_info_score(pred, data.y.cpu())
print('[INFO]NMI: ', nmi)

ami = adjusted_mutual_info_score(pred, data.y.cpu())
print('[INFO]AMI: ', ami)

In [ ]:

pred = kmeans.fit_predict(x.cpu())

nmi = normalized_mutual_info_score(pred, data.y.cpu())
print('[INFO]NMI: ', nmi)

ami = adjusted_mutual_info_score(pred, data.y.cpu())
print('[INFO]AMI: ', ami)